In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns

sns.set_style("darkgrid")

In [ ]:
"""
Update parameters
"""
COLLECTION = "MekaVerse"
ST_DEVS = 3 # set number of st devs away from mean offer
MINT_PRICE = 0.2

"""
Plot params
"""
plt.rcParams["figure.figsize"] = [15, 5]
plt.rcParams["figure.autolayout"] = True


"""
Import pre-reveal sales and bids
"""
bids = pd.read_csv(f"pre-reveal_bids/{COLLECTION}_pre-reveal_bids.csv")
sales = pd.read_csv(f"pre-reveal_sales/{COLLECTION}_pre-reveal_sales.csv")


"""
Helper functions
"""
def get_mean(offers):
    return np.mean(offers)

def get_st_dev(offers):
    return np.std(offers)

def find_anomalous_sale(sale_price, mean, std):
    if sale_price > (mean + ST_DEVS * std):
        return True
    else:
        return False
    
def percentage_change(col1, col2):
    return ((col2 - col1) / col1) * 100

In [ ]:
"""
Print overpaid tokens relative to mean offer price
"""

overpaid_tokens = list()

for token in bids["TOKEN_ID"].unique():
    offers = bids[bids["TOKEN_ID"] == token]["OFFER"]
    if len(offers) > 5:
        mean = get_mean(offers)
        std = get_st_dev(offers)
        if len(sales[sales["TOKEN_ID"] == token]) > 0:
            sale_price = sales[sales["TOKEN_ID"] == token].iloc[0]["PRICE"]
            rank = sales[sales["TOKEN_ID"] == token].iloc[0]["RANK"]
            user = sales[sales["TOKEN_ID"] == token].iloc[0]["USER"]
            if find_anomalous_sale(sale_price, mean, std):
                print(f"Token: {token}, Mean Offer: {mean}, Sale Price: {sale_price}, Rank: {rank}, User: {user}")
                overpaid_tokens.append(token)
        else:
            continue

In [ ]:
"""
Show distribution of offers and sale prices
"""

for token in overpaid_tokens:
    
    token_bids = bids[bids["TOKEN_ID"] == token].copy()
    token_sales = sales[sales["TOKEN_ID"] == token].copy()

    ax = sns.kdeplot(token_bids["OFFER"], fill=True)

    xmin, xmax, ymin, ymax = plt.axis()

    for index, row in token_sales.iterrows():
        plt.plot([row["PRICE"], row["PRICE"]], [0, ymax], 'r', label="SALE PRICE", linestyle="dashed")
    
    plt.title(token)
    plt.show()

In [ ]:
"""
Show offers that are significantly above mint price
"""

bids["MINT"] = MINT_PRICE
bids["OVER_MINT_PERC"] = percentage_change(bids["MINT"], bids["OFFER"])
bids.sort_values(by="OVER_MINT_PERC", ascending=False, inplace=True)
bids.head(40)